In [25]:
!pip install html2image

# Start Here

In [26]:
import json
import os
import random
import re
import textwrap

from html2image import Html2Image
from nltk.corpus import stopwords, wordnet
from nltk.metrics.distance import edit_distance
import pytrends
import nltk
from tqdm import tqdm

from sumy.nlp.stemmers import Stemmer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer

In [27]:
stopwords = stopwords.words('english')

In [28]:
with open('ppn_deck.json', 'r') as read_file:
    card_deck = json.load(read_file)

with open('card_html.txt', 'r') as f:
    html_template = f.read()


In [34]:
def summarize_text(text, num_sentences):
    """
    Summarize the given text using the LSA or LexRank summarization algorithms and return the summary as a string
    """
    # create a PlaintextParser object to parse the text
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    # choose a summarization algorithm
    # algorithm = LsaSummarizer()
    algorithm = LexRankSummarizer()

    # summarize the text and return the summary as a string
    summary = algorithm(parser.document, num_sentences)
    summary_text = "\n".join([str(sentence) for sentence in summary])

    return summary_text

def generate_card(title: str, description: str, points: str, html_template,  name):
    """
    generate_card generates a card image from the given title, description, and points

    :param title: _description_
    :type title: str
    :param description: _description_
    :type description: str
    :param points: _description_
    :type points: str
    """
    # Create the HTML content for the card
    # The card is 550x850 pixels
    # read the template file from the current directory 'card_html.txt'
    # cast the points as int then str to remove the decimal point
    points = int(points) # removes the decimal point
    points = str(points) # now it's a string
    # replace the placeholders with the actual values
    html = html_template
    html = html.replace('Card Title', title)
    html = html.replace('Card Description', description)
    html = html.replace('Point Value', str(points))
    space_to_fill = str(850 - len(description) - 100)
    html = html.replace('MIDDLE_HEIGHT',str(space_to_fill))
    # create blank lines to fill the space
    Middle_Content = ' '*int(space_to_fill)
    html = html.replace('Middle Content',str(Middle_Content))

    # Create an Html2Image object
    hti = Html2Image()

    # Generate a screenshot of the HTML content and save it to a file
    title = title.replace(' ', '_')
    output_path = './card_box/'
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    filename = '{}.png'.format(title)
    savepath = os.path.join(output_path, filename)
    hti.output_path = output_path
    hti.screenshot(html_str=html, save_as=filename)

def generate_physical_cards():
    # Choose a random card to print
    card = random.choice(card_deck)
    print(card)

    # Prepare the summary and points variables
    summary = card['summary_short']
    if isinstance(summary, list):
        summary = summary[1]
    #!summary = summarize_text(summary, 3)
    if isinstance(summary, list):
        summary = ' '.join(summary)
    
    # points are defined in the json file under the key 'points_for_card'
    points = card['points_for_card']

    # Generate the card
    # With this:
    generate_card(str(card['title']), summary, points=points, html_template=html_template, name=str(card['title']))

    # Iterate through each card and generate a card image for it
    for card in tqdm(card_deck):
        # Prepare the summary and points variables
        summary = card['summary']
        if isinstance(summary, list):
            summary = summary[1]
        summary = summarize_text(summary, 2)
        if isinstance(summary, list):
            summary = ' '.join(summary)
        points = len(set(summary.split()))

        # Generate the card
        generate_card(str(card['title']), summary, points=points, html_template=html_template, name=str(card['title']))

print("Initialized process, and ready to generate physical cards...")


Initialized process, and ready to generate physical cards...


In [35]:
generate_physical_cards()
print("Done")

{'title': 'Jean Baudrillard', 'summary': 'Jean Baudrillard, Jean Baudrillard UK: BOHD-rih-yar, US: BOHD-ree-AR, French: bodija; 27 July 1929 6 March 2007 was a French sociologist, philosopher and poet with interest in cultural studies. He is best known for his analyses of media, contemporary culture, and technological communication, as well as his formulation of concepts such as simulation and hyperreality. Baudrillard wrote about diverse subjects, including consumerism, gender relations, critique of economy, economics, social history, art, Western foreign policy, and popular culture. Among his best known works are Seduction 1978, Simulacra and Simulation 1981, America 1986, and The Gulf War Did Not Take Place 1991. His work is frequently associated with postmodernism and specifically post-structuralism. Nevertheless, Baudrillard has also opposed post-structuralism and had distanced himself from postmodernism.', 'related': 817, 'summary_short': 'UK: BOHD-rih-yar, US: BOHD-ree-AR, Frenc

objc[29215]: Class WebSwapCGLLayer is implemented in both /System/Library/Frameworks/WebKit.framework/Versions/A/Frameworks/WebCore.framework/Versions/A/Frameworks/libANGLE-shared.dylib (0x7ffa59435aa0) and /Applications/Google Chrome.app/Contents/Frameworks/Google Chrome Framework.framework/Versions/108.0.5359.124/Libraries/libGLESv2.dylib (0x112aab568). One of the two will be used. Which one is undefined.
[1227/114813.423200:INFO:headless_shell.cc(623)] Written to file /Users/grahamwaters/Library/Mobile Documents/com~apple~CloudDocs/Mimikers/card_box/Jean_Baudrillard.png.
  0%|          | 0/9296 [00:00<?, ?it/s]objc[29259]: Class WebSwapCGLLayer is implemented in both /System/Library/Frameworks/WebKit.framework/Versions/A/Frameworks/WebCore.framework/Versions/A/Frameworks/libANGLE-shared.dylib (0x7ffa59435aa0) and /Applications/Google Chrome.app/Contents/Frameworks/Google Chrome Framework.framework/Versions/108.0.5359.124/Libraries/libGLESv2.dylib (0x116fae568). One of the two will b

KeyboardInterrupt: 